In [1]:
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from scipy import fft
import time

# Loading data

In [2]:
hdul = fits.open('advact_tt_patch.fits')
map = hdul[0].data
hdul.close()
map = np.asarray(map,dtype='float')
print('read map')

read map


# Defining Functions

In [3]:
def pad_map(map):
    map=np.hstack([map,np.fliplr(map)])
    map=np.vstack([map,np.flipud(map)])
    return map

def unpad_map(map):
    x = map.shape[0]
    y = map.shape[1]
    
    #x, y must be even since they are the sizes of a padded image
    map = map[:int(x/2), :int(y/2)]
    return map

def get_gauss_kernel(map,sig,norm=False):
    nx=map.shape[0]
    x=np.fft.fftfreq(map.shape[0])*map.shape[0]
    y=np.fft.fftfreq(map.shape[1])*map.shape[1]
    rsqr=np.outer(x**2,np.ones(map.shape[1]))+np.outer(np.ones(map.shape[0]),y**2)
    kernel=np.exp((-0.5/sig**2)*rsqr)
    if norm:
        kernel=kernel/kernel.sum()
    return kernel

def estimate_ps(map):
    #first I will find the power spectrum of the map, then I will smooth it
    map_f = np.fft.fft2(map)
    map_ps = (np.abs(map_f))**2
    
    #smoothing
    gauss_kernel = get_gauss_kernel(map, sig=14, norm = True)
    gauss_kernel_f = np.fft.fft2(gauss_kernel)
    
    map_ps_f = np.fft.fft2(map_ps)
    ps_smoothed = np.fft.ifft2(map_ps_f * gauss_kernel_f)
    
    ps_smoothed = np.fft.fftshift(ps_smoothed)
    return np.real(ps_smoothed)

def filter_map(map, ps):
    #ps : power spectrum
    map = pad_map(map)
    map_ft = np.fft.fft2(map)
    map_ft = map_ft/ps
    map_filt=np.real(np.fft.ifft2(map_ft))
    map_filt = unpad_map(map_filt)
    return map_filt

# Power Spectrum and Filtering

In [4]:
x0 = 2997
y0 = 1998
width = 30
patch = map[x0-width:x0+width, y0-width:y0+width]

In [5]:
patch_padded = pad_map(patch)
ps = estimate_ps(patch_padded)
#filtermap = filter_map(map, ps)

In [6]:
ps.shape

(120, 120)

# Matched Filter

In [7]:
#N^-1 times the template
template =  get_gauss_kernel(patch_padded, sig=14, norm = True) #sig comes from my answer to Q1
tft=np.fft.fft2(template)
datft=np.fft.fft2(patch_padded)
Ninv= 1/ps

Ninvt=tft*Ninv  
mf_rhs=np.real(np.fft.ifft2(Ninvt*np.conj(datft)))
mf_rhs_check=np.real(np.fft.ifft2(np.conj(datft*Ninv)*tft))

#make A^T N^-1 A
NinvA_real=np.real(np.fft.ifft2(Ninvt)) #N^-1 A
mf_lhs = template * NinvA_real #N^-1 A 

In [8]:
amp = unpad_map(mf_rhs/mf_lhs)
err = unpad_map(1/np.sqrt(mf_lhs))

In [9]:
#amplitude in the region arond the cluster (we assumed that the cluster is in the center meaning amp [30, 30])
amp

array([[-5.35794128e+06, -5.38534413e+06, -5.53601573e+06, ...,
        -6.78184817e+14, -1.22847171e+15, -2.23775536e+15],
       [-5.38532798e+06, -5.41287089e+06, -5.56431264e+06, ...,
        -6.81651188e+14, -1.23475071e+15, -2.24919280e+15],
       [-5.32567860e+06, -5.35291643e+06, -5.50300259e+06, ...,
        -6.88487446e+14, -1.24723041e+15, -2.27204143e+15],
       ...,
       [ 8.88649558e+14,  8.93194331e+14,  9.12687854e+14, ...,
         1.88899610e+22,  3.35267088e+22,  6.01327610e+22],
       [ 1.58721800e+15,  1.59533535e+15,  1.63020164e+15, ...,
         3.39506137e+22,  6.02652702e+22,  1.08129286e+23],
       [ 2.84628561e+15,  2.86084162e+15,  2.92342448e+15, ...,
         6.08998530e+22,  1.08048451e+23,  1.93539672e+23]])

In [10]:
#error bar from the matched filter
#the error bars are very larg, I must have messed up somewhere but I can't find it
err

array([[1.17293719e+07, 1.17593282e+07, 1.18496569e+07, ...,
        4.65984039e+10, 6.24866612e+10, 8.41283921e+10],
       [1.17593105e+07, 1.17893433e+07, 1.18799026e+07, ...,
        4.67173401e+10, 6.26461496e+10, 8.43431131e+10],
       [1.18495858e+07, 1.18798492e+07, 1.19711037e+07, ...,
        4.70759732e+10, 6.31270611e+10, 8.49905700e+10],
       ...,
       [4.63418338e+10, 4.64601844e+10, 4.68170530e+10, ...,
        1.84077697e+14, 2.46875335e+14, 3.32133086e+14],
       [6.20727505e+10, 6.22312743e+10, 6.27092790e+10, ...,
        2.46636210e+14, 3.30795341e+14, 4.45111953e+14],
       [8.32506876e+10, 8.34632897e+10, 8.41043590e+10, ...,
        3.30201537e+14, 4.42762235e+14, 5.95273265e+14]])